In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
from sklearn.model_selection import train_test_split

# Set reproducibility
np.random.seed(42)

# Simulation parameters
N_total = 1000000       # total population
N_train = 800000       # training cohort
N_test = N_total - N_train
D = 6                # number of variables
bins_list = [4, 8, 16, 36, 64, 128]  # discretization levels

# Generate correlated multivariate data
mu = np.zeros(D)
Sigma = 0.5 * np.ones((D, D)) + 0.5 * np.eye(D)  # correlated structure
latent = np.random.multivariate_normal(mu, Sigma, size=N_total)

# Split train/test
latent_train = latent[:N_train]
latent_test  = latent[N_train:]

# Assume the first variable is the clinical outcome
clinical_outcome = latent[:, 0]

def quantile_discretize(arr, bins):
    """Discretize each column into equal-frequency quantile bins."""
    arr_disc = np.zeros_like(arr, dtype=int)
    for j in range(arr.shape[1]):
        ranks = arr[:, j].argsort().argsort()
        arr_disc[:, j] = np.ceil((ranks + 1) / len(ranks) * bins).astype(int)
        arr_disc[arr_disc[:, j] > bins, j] = bins
    return arr_disc

def identify_accuracy(train, test, bins):
    """Estimate identifiability for given discretization."""
    mu_hat = train.mean(axis=0)
    Sigma_hat = np.cov(train, rowvar=False)
    inv_Sigma = np.linalg.pinv(Sigma_hat)
    
    correct = 0
    known_var = 0  # assume we only know variable 0 for reidentification

    for i, x in enumerate(test):
        x_obs = x[known_var]
        likelihoods = []
        for j, candidate in enumerate(test):
            diff = candidate - mu_hat
            mdist = diff @ inv_Sigma @ diff.T
            likelihoods.append(-mdist)
        guess = np.argmax(likelihoods)
        if guess == i:
            correct += 1
    return correct / len(test)

# Run for each discretization level
results = []
for bins in bins_list:
    print(f"\n=== Evaluating discretization with {bins} bins ===")
    disc = quantile_discretize(latent, bins)
    train_disc = disc[:N_train]
    test_disc  = disc[N_train:]
    acc = identify_accuracy(train_disc, test_disc, bins)
    print(f"Identifiability (Top-1 Accuracy): {acc*100:.2f}%")
    results.append((bins, acc))

df = pd.DataFrame(results, columns=["Discretization_bins", "Identifiability"])
print(df)